Download Dataset

In [ ]:
! pip install --user kaggle
! sudo apt install unzip

In [ ]:
! mkdir /root/.kaggle
! touch /root/.kaggle/kaggle.json
! echo '{"username":"----","key":"----"}' > /root/.kaggle/kaggle.json
! kaggle datasets download lexset/synthetic-asl-alphabet

In [ ]:
! unzip synthetic-asl-alphabet.zip

Import Necessary Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import os
from tqdm import tqdm
import string

Prepare Images

In [3]:
def rotate(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  rot_img = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return rot_img

In [4]:
img_size = 100
data_dir = "Train_Alphabet/"
X = []
y = []
i = -1
alphabet = list(string.ascii_uppercase)
alphabet.append('Blank')
for folder in alphabet:
    print(folder)
    i+=1
    img_num = -20
    for img in tqdm(os.listdir(data_dir+folder)):
        img_num += 1
        if img_num > 20:
          img_num = -20
        img_file = cv2.imread(data_dir+folder+'/'+img)
        img_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_file, (img_size, img_size))
        img_resized = rotate(img_resized, img_num)
        X.append(img_resized)
        y.append(i)
X = np.asarray(X)
y = np.asarray(y)

A


100%|██████████| 900/900 [00:15<00:00, 58.11it/s] 


B


100%|██████████| 900/900 [00:08<00:00, 100.82it/s]


C


...

100%|██████████| 900/900 [00:09<00:00, 96.51it/s] 


Z


100%|██████████| 900/900 [00:09<00:00, 96.84it/s] 


Blank


100%|██████████| 900/900 [00:09<00:00, 95.59it/s]


In [5]:
x_norm=tf.keras.utils.normalize(X, axis=1)

CNN

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.15, shuffle= True)


model = Sequential()
model.add(Conv2D(filters=100, input_shape=(img_size,img_size,1), kernel_size=(11,11), strides=(2,2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Conv2D(filters=150, kernel_size=(3, 3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
model.add(Flatten())

model.add(Dense(200))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.7))

model.add(Dense(150))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.7))

#2nd Fully Connected Layer
model.add(Dense(27))
model.add(BatchNormalization())
model.add(Activation('softmax'))


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 50, 100)       12200     
                                                                 
 batch_normalization (BatchN  (None, 50, 50, 100)      400       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 50, 50, 100)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 100)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 25, 150)       135150    
                                                                 
 batch_normalization_1 (Batc  (None, 25, 25, 150)      6

Train Neural Net

In [10]:
model.fit(x_train, y_train, batch_size = 40, epochs = 40, validation_freq=1, validation_data=(x_test, y_test), verbose=1)

Epoch 1/40
517/517 [==============================] - 14s 26ms/step - loss: 0.4551 - accuracy: 0.8733 - val_loss: 0.2565 - val_accuracy: 0.9372
Epoch 2/40
517/517 [==============================] - 13s 25ms/step - loss: 0.4272 - accuracy: 0.8824 - val_loss: 0.5063 - val_accuracy: 0.8741
Epoch 3/40
517/517 [==============================] - 13s 25ms/step - loss: 0.4076 - accuracy: 0.8872 - val_loss: 0.3267 - val_accuracy: 0.9177
Epoch 4/40
517/517 [==============================] - 13s 25ms/step - loss: 0.3920 - accuracy: 0.8909 - val_loss: 0.2799 - val_accuracy: 0.9289
Epoch 5/40
517/517 [==============================] - 13s 25ms/step - loss: 0.3882 - accuracy: 0.8893 - val_loss: 0.2180 - val_accuracy: 0.9457
Epoch 6/40
517/517 [==============================] - 13s 25ms/step - loss: 0.3814 - accuracy: 0.8964 - val_loss: 0.4016 - val_accuracy: 0.8925
Epoch 7/40
517/517 [==============================] - 13s 26ms/step - loss: 0.3645 - accuracy: 0.8990 - val_loss: 0.2218 - val_accuracy:

Evaluation

In [14]:
results = model.evaluate(x_test, y_test)
print('test accuracy:', results[1])

114/114 [==============================] - 1s 11ms/step - loss: 0.1665 - accuracy: 0.9553
test accuracy: 0.9552811980247498


In [11]:
 model.save('model.h5')